# MEWpy Community Modeling

Author: Vitor Pereira, inspired on the work by Daniel Machado. 

License: [CC BY-SA 4.0](http://creativecommons.org/licenses/by-sa/4.0/)

-------

In this tutorial:

- You will learn how to perform flux balance analysis of microbial communities
using a model of the [central carbon metabolism of *E. coli*](https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1).


## Install requirements 
To run this notebook we firstly need to install the required packages

In [1]:
! pip install -U -q mewpy cplex escher

Verify the instalation

In [2]:
import mewpy
mewpy.info()

MEWpy version: 0.1.36
Author: Vitor Pereira (2019-) and CEB University of Minho (2019-2023)
Contact: vpereira@ceb.uminho.pt 

Available LP solvers: gurobi cplex glpk
Default LP solver: cplex 

Available ODE solvers: scikits scipy
Default ODE solver: scikits 

Optimization Problems: AbstractKOProblem AbstractOUProblem CofactorSwapProblem CommunityKOProblem ETFLGKOProblem ETFLGOUProblem GKOProblem GOUProblem GeckoKOProblem GeckoOUProblem KcatOptProblem KineticKOProblem KineticOUProblem MediumProblem OptORFProblem OptRamProblem RKOProblem ROUProblem 

Available EA engines: inspyred jmetal
Default EA engine: jmetal
Available EAs: GA NSGAII NSGAIII SA SPEA2 



**IMPORTANT**: The notebook requires a MEWpy version >= 0.1.36

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)

### Run in Google colab

If you are running this notebook in Colab, you need to perform the following steps, otherwise skip.

In [4]:
%%bash
[[ ! -e /colabtools ]] && exit
! pip install -U -q PyDrive

In [5]:
if 'google.colab' in str(get_ipython()):
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    model_file = drive.CreateFile({'id':'1o0XthuEOs28UJ4XTa9SfFSFofazV-2nN'})
    model_file.GetContentFile('e_coli_core.xml.gz')

## Setting up a community

We will create a synthetic microbial consortium with two *E. coli* mutants growing in minimal medium. In one of the mutants we will knockout the glucose transporter and in the other we will knockout the ammonium transporter.

In [6]:
from cobra.io import read_sbml_model
from mewpy import get_simulator

model = read_sbml_model('models/ec/e_coli_core.xml.gz')
wildtype = get_simulator(model)
solution = wildtype.simulate()
print(solution)
solution.find('EX')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-11
objective: 0.8739215069684301
Status: OPTIMAL
Method:FBA


,Flux rate
Reaction ID,
EX_co2_e,22.809833
EX_glc__D_e,-10.000000
EX_h_e,17.530865
EX_h2o_e,29.175827
EX_nh4_e,-4.765319
EX_o2_e,-21.799493
EX_pi_e,-3.214895


Now we create our two mutants (`glc_ko` and `nh4_ko`):

In [7]:
glc_ko = wildtype.copy()
glc_ko.id = 'glc_ko'
glc_ko.set_reaction_bounds('GLCpts', 0, 0)

In [8]:
nh4_ko = wildtype.copy()
nh4_ko.id = 'nh4_ko'
nh4_ko.set_reaction_bounds('NH4t', 0, 0)

## Comparing models

Community models require that metabolites have the same identifiers accros all models. MEWpy offers some functions tho that end, computing the metabolites, reactions and uptakes overlaps between a list models.

In [9]:
from mewpy.com import *
mets, rxns, over = jaccard_similarity_matrices([glc_ko, nh4_ko])

In [10]:
mets

,glc_ko,nh4_ko
glc_ko,1.0,1.0
nh4_ko,1.0,1.0


In [11]:
rxns

,glc_ko,nh4_ko
glc_ko,1.000000,0.978947
nh4_ko,0.978947,1.000000


In [12]:
over

,glc_ko,nh4_ko
glc_ko,1.0,1.0
nh4_ko,1.0,1.0


## Building communities

**MEWpy** has some basic functionality for working with microbial communities, one is the `CommunityModel` class to create microbial communities from a list of models of individual species: 

In [13]:
from mewpy.model import CommunityModel
community = CommunityModel([glc_ko, nh4_ko], merge_biomasses=False, flavor='cobra')

Arguments:
- `merge_biomasses=False`: The model does not assume a relative abundance of organisms in the community. If set to `True`, a default abundance of 1:1 would be assumed.
- `flavor='cobra'`: The model is built over the 'Cobra' framework. `reframed` is an alternative option which is faster for large communities. 

Additional optional arguments:

- `abundances`: A list of relative abundances for each model (use the same order of the models), e.g, `abundances=[1,2]`.  
                    
- `add_compartments`: If each organism external compartment is to be added to the community model. Default `True`.
- `balance_exchanges`: If the organisms uptakes should reflect their abundances. This will normalize each organism flux value in acordance to the abundance. Default `True`.  
- `copy_models`: if the models are to be copied, default `True`.
  

In [14]:
sim = community.get_community_model()

Organism: 100%|███████████████████████████████████| 2/2 [00:00<00:00,  8.69it/s]


This community model ignores the environmental conditions that were specified in the original models (since these could be very different). 

To make our life easier, we will extract the nutrient composition specified in the wild-type model to use later.

In [15]:
from mewpy.simulation import Environment
M9 = Environment.from_model(wildtype)
M9

,lb,ub
EX_ac_e,0.0,1000.0
EX_acald_e,0.0,1000.0
EX_akg_e,0.0,1000.0
EX_co2_e,-1000.0,1000.0
EX_etoh_e,0.0,1000.0
EX_for_e,0.0,1000.0
EX_fru_e,0.0,1000.0
EX_fum_e,0.0,1000.0
EX_glc__D_e,-10.0,1000.0
EX_gln__L_e,0.0,1000.0


## Simulation using FBA

A very simple way to simulate a microbial community is to merge the individual models into a single model that mimics a "super organism", where each microbe lives inside its own compartment, and run a (conventional) FBA simulation for this *super organism*.

In [16]:
solution = sim.simulate(constraints=M9)

print(solution)
solution.find('EX')

objective: 0.831195550185812
Status: OPTIMAL
Method:FBA


,Flux rate
Reaction ID,
EX_ac_e_glc_ko,-6.134506
EX_akg_e_glc_ko,-4.532343
EX_co2_e_glc_ko,9.064686
EX_etoh_e_glc_ko,1.602163
EX_glu__L_e_glc_ko,4.532343
EX_h_e_glc_ko,-6.134506
EX_h2o_e_glc_ko,7.462523
EX_nh4_e_glc_ko,-4.532343
EX_o2_e_glc_ko,-5.196352


We can see that the model predicts a growth rate (total biomass per hour) similar to the wild-type, with an efficient consumption of glucose and ammonia that results in respiratory metabolism.

But what is each organism doing, and are both organisms actually growing at the same rate?

Let's print the biomass flux for each organism:

In [17]:
solution.find('BIOMASS', sort=True,show_nulls=True)

,Flux rate
Reaction ID,
BIOMASS_Ecoli_core_w_GAM_glc_ko,0.000000
BIOMASS_Ecoli_core_w_GAM_nh4_ko,0.831196


and all non null fluxes by organism:

In [18]:
sim.find_metabolites()

,name,compartment,formula
id,,,
community_biomass,Total community biomass,e,None
glc__D_e_glc_ko,D-Glucose,e_glc_ko,C6H12O6
glc__D_e,D-Glucose,e,C6H12O6
gln__L_c_glc_ko,L-Glutamine,c_glc_ko,C5H10N2O3
gln__L_e_glc_ko,L-Glutamine,e_glc_ko,C5H10N2O3
...,...,...,...
fru_e_nh4_ko,D-Fructose,e_nh4_ko,C6H12O6
fum_c_nh4_ko,Fumarate,c_nh4_ko,C4H2O4
fum_e_nh4_ko,Fumarate,e_nh4_ko,C4H2O4


In [19]:
solution.find('glc_ko')

,Flux rate
Reaction ID,
ACALD_glc_ko,-1.602163
ACKr_glc_ko,6.134506
ACONTa_glc_ko,4.532343
ACONTb_glc_ko,4.532343
ACt2r_glc_ko,6.134506
AKGDH_glc_ko,4.532343
AKGt2r_glc_ko,4.532343
ALCD2x_glc_ko,-1.602163
ATPM_glc_ko,8.390000


We can look at the organisms' exchanges with the environment

In [20]:
exchanges(community,solution)

,glc_ko,nh4_ko,Total
Metabolite,,,
glc__D_e,0.000000,-10.000000,-10.00000
gln__L_e,0.000000,0.000000,0.00000
glu__L_e,4.532343,-4.532343,0.00000
h2o_e,7.462523,23.220295,30.68282
h_e,-6.134506,22.808289,16.67378
lac__D_e,0.000000,0.000000,0.00000
mal__L_e,0.000000,0.000000,0.00000
nh4_e,-4.532343,0.000000,-4.53234
o2_e,-5.196352,-18.470761,-23.66711


Actually it seems that only one of the organisms is growing while the other has an active metabolism (it exchanges metabolites with the environment and with the other organism) performing the role of a bioconverter, but none of the flux is used for growth. 

> Do you think this would be a stable consortium ?

## Regularized Community FBA

Flux balance analysis (FBA) provides not one but potentially an infinite number of solutions. There are, however, a number different strategies to select one particular solution from the set of all possibles. One common approach is to select to most parsimonious solution that minimizes the sum of all fluxes (pFBA). 

Next, we simulate the community growth and select a solution based on L2 regularization of each community organism growth, that is, we aim to find the solution for which no organisms growth to fast while approaching each organisms individual growth:   

In [21]:
from mewpy.com import regComFBA

In [22]:
solution = regComFBA(community,constraints=M9,obj_frac=1)
solution.find('BIOMASS|growth', sort=True, show_nulls=True)

,Flux rate
Reaction ID,
BIOMASS_Ecoli_core_w_GAM_glc_ko,4.545680e-11
BIOMASS_Ecoli_core_w_GAM_nh4_ko,8.311956e-01
community_growth,8.311956e-01


Or using the simulator

In [23]:
solution=sim.simulate(method=regComFBA,constraints=M9)
solution.find('BIOMASS|growth', sort=True, show_nulls=True)

,Flux rate
Reaction ID,
BIOMASS_Ecoli_core_w_GAM_glc_ko,0.227885
BIOMASS_Ecoli_core_w_GAM_nh4_ko,0.594999
community_growth,0.822884


By default, regComFBA considers a confidence on community growth of 99%. However, you can relax this constraint to a lower value, e.g., `obj_frac=0.9` (90%)

In [24]:
solution=sim.simulate(method=regComFBA,constraints=M9,obj_frac=0.9)
solution.find('BIOMASS|growth', sort=True, show_nulls=True)

,Flux rate
Reaction ID,
BIOMASS_Ecoli_core_w_GAM_glc_ko,0.374038
BIOMASS_Ecoli_core_w_GAM_nh4_ko,0.374038
community_growth,0.748076


## Community Simulation with SteadyCom

**SteadyCom** by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) is a recent community simulation method that takes into account the fact that to reach a stable composition the organisms need to grow at the same *specific growth rate* (1/h), which means that the *absolute growth rate* (gDW/h) of each organism is proportional to its *abundance* at steady-state (gDW).

Let's simulate the same community using SteadyCom:

In [25]:
solution = SteadyCom(community, constraints=M9)

Organism: 100%|███████████████████████████████████| 2/2 [00:00<00:00,  8.91it/s]


In this case the solution object shows the overall community growth rate and the relative abundance of each species:

In [26]:
solution

Community growth: 0.873046875
glc_ko	0.024884385078816997
nh4_ko	0.9751156149211829

The `solution` object for community simulations implements a few additional features, such as enumerating all the cross-feeding interactions:

In [27]:
solution.cross_feeding(as_df=True).dropna().sort_values('rate', ascending=False)

,donor,receiver,compound,rate
12,nh4_ko,glc_ko,pyr_e,24.884385
6,glc_ko,nh4_ko,lac__D_e,24.721311
18,nh4_ko,glc_ko,etoh_e,12.124982
14,glc_ko,nh4_ko,acald_e,9.701650
15,nh4_ko,glc_ko,akg_e,4.704342
1,glc_ko,nh4_ko,glu__L_e,4.642087
13,nh4_ko,glc_ko,ac_e,2.696120
2,nh4_ko,glc_ko,h2o_e,2.610785
4,nh4_ko,glc_ko,h_e,2.547897


We can plot the fluxes of each mutant in a map to help with interpretation of the results:

In [28]:
from mewpy.visualization.escher import build_escher
if 'google.colab' in str(get_ipython()):
    from google.colab import output
    output.enable_custom_widget_manager()

build_escher(fluxes=solution.internal['glc_ko'])

Builder(reaction_data={'ACALD': 2.4233312762522647, 'ACALDt': -9.701650317231756, 'ACKr': 2.6961195998247263, …

In [29]:
build_escher(fluxes=solution.internal['nh4_ko'])

Builder(reaction_data={'ACALD': -2.4233312762522647, 'ACALDt': 9.701650317231756, 'ACKr': -2.6961195998247263,…

## Explore alternative solutions

Unfortunately, one limitation of **SteadyCom**, which is exemplified by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) in Figure 3 (reproduced below), is the variability in the solution space when the community is not growing at the maximum (theoretical) growth rate.

> Would you expect a synthetic community to grow at its maximum growth rate?

**MEWpy** implements a variability analysis function for the SteadyCom solution space, let's see what happens if the community is growing at 90% of the theoretical maximum:

In [30]:
from mewpy.com import SteadyComVA
variability = SteadyComVA(community, obj_frac=0.9, constraints=M9)

print('Strain\tMin\tMax')
for strain, (lower, upper) in variability.items():
    print(f'{strain}\t{lower:.1%}\t{upper:.1%}')

Strain	Min	Max
glc_ko	0.4%	98.3%
nh4_ko	1.7%	99.6%


As you can see, there is a really large variability in this solution space. This means that we know in theory the two mutants **can** cooperate and survive in minimal media, but there is still a lot of uncertainty with regard to **how** they will achieve a stable consortium.

> How do you think we can reduce this uncertainty?

Firstly, lets set the environment conditions:

In [31]:
sim.set_environmental_conditions(M9)

We may now impose constraints on each organism growth, such as stating that each organism need to grow at least 0.1/h

In [32]:
constraints={community.organisms_biomass['nh4_ko']:(0.1,1000), 
             community.organisms_biomass['glc_ko']:(0.1,1000)}
solution = sim.simulate(constraints=constraints)
solution

objective: 0.8283090782473191
Status: OPTIMAL
Method:FBA

In [33]:
solution.find('BIOMASS')

,Flux rate
Reaction ID,
BIOMASS_Ecoli_core_w_GAM_glc_ko,0.100000
BIOMASS_Ecoli_core_w_GAM_nh4_ko,0.728309


Alternatively, we might choose to impose relative growth rates for each of the organisms, as a proxy of the community composition:

In [34]:
community = CommunityModel([glc_ko, nh4_ko],
                           add_compartments=True,
                           merge_biomasses=True,
                           flavor='cobra')

In [35]:
sim = community.get_community_model()
sim.set_environmental_conditions(M9)

Organism: 100%|███████████████████████████████████| 2/2 [00:00<00:00,  8.61it/s]


In [36]:
solution = sim.simulate()
print(solution)
solution.find('BIOMASS')

objective: 0.4075720936398621
Status: OPTIMAL
Method:FBA


,Flux rate
Reaction ID,
BIOMASS_Ecoli_core_w_GAM_glc_ko,0.407572
BIOMASS_Ecoli_core_w_GAM_nh4_ko,0.407572


In [37]:
sim.find(community.biomass)

,name,lb,ub,stoichiometry,gpr,annotations
id,,,,,,
community_growth,Community growth rate,0,inf,"{'Biomass_glc_ko': -1, 'Biomass_nh4_ko': -1}",,{}


The relative abundance (relative growth rates) are by default equal. We may though change these ratios:  

In [38]:
community.set_abundance({'glc_ko':1,'nh4_ko':2.5})
sim.simulate(method='pFBA').find('BIOMASS|growth')

,Flux rate
Reaction ID,
BIOMASS_Ecoli_core_w_GAM_glc_ko,0.105388
BIOMASS_Ecoli_core_w_GAM_nh4_ko,0.263471
community_growth,0.105388


## SMETANA

**SMETANA** implements several algorithms to analyse cross-feeding interactions in microbial communities. These have been describe in [Zelezniak et al, PNAS (2015)](https://www.pnas.org/doi/abs/10.1073/pnas.1421834112). Please read the paper for a more detailed explanation.

### Species Coupling Score
**SCS** (species coupling score): measures the dependency of one species in the presence of the others to survive

In [39]:
sc_score(community)

Organism: 100%|███████████████████████████████████| 2/2 [00:00<00:00,  8.89it/s]


,Value
Attribute,
glc_ko,{'nh4_ko': 1.0}
nh4_ko,{'glc_ko': 1.0}


### Metabolite Uptake Score
**MUS** (metabolite uptake score): measures how frequently a species needs to uptake a metabolite to survive

In [40]:
MUS = mu_score(community)
MUS

,Value
Attribute,
glc_ko,"{'ac_e': 0.18, 'acald_e': 0.26, 'akg_e': 0.1, ..."
nh4_ko,"{'ac_e': 0.07692307692307693, 'acald_e': 0.076..."


In [41]:
MUS.glc_ko

{'ac_e': 0.18,
 'acald_e': 0.26,
 'akg_e': 0.1,
 'co2_e': 0.0,
 'etoh_e': 0.39,
 'for_e': 0.0,
 'fru_e': 0.0,
 'fum_e': 0.0,
 'glc__D_e': 0.0,
 'gln__L_e': 0.0,
 'glu__L_e': 0.0,
 'h_e': 0.09,
 'h2o_e': 0.13,
 'lac__D_e': 0.25,
 'mal__L_e': 0.0,
 'nh4_e': 1.0,
 'o2_e': 0.89,
 'pi_e': 1.0,
 'pyr_e': 0.27,
 'succ_e': 0.03}

In [42]:
MUS.nh4_ko

{'ac_e': 0.07692307692307693,
 'acald_e': 0.07692307692307693,
 'akg_e': 0.0,
 'co2_e': 0.0,
 'etoh_e': 0.0,
 'for_e': 0.0,
 'fru_e': 0.0,
 'fum_e': 0.0,
 'glc__D_e': 1.0,
 'gln__L_e': 0.0,
 'glu__L_e': 1.0,
 'h_e': 0.0,
 'h2o_e': 0.0,
 'lac__D_e': 0.0,
 'mal__L_e': 0.0,
 'nh4_e': 0.0,
 'o2_e': 0.07692307692307693,
 'pi_e': 1.0,
 'pyr_e': 0.07692307692307693,
 'succ_e': 0.0}

### Metabolite Production Score
**MPS** (metabolite production score): measures the ability of a species to produce a metabolite

In [43]:
MPS = mp_score(community,environment=M9)
MPS

,Value
Attribute,
glc_ko,"{'etoh_e': 1, 'for_e': 1, 'h2o_e': 1, 'pyr_e':..."
nh4_ko,"{'etoh_e': 1, 'for_e': 1, 'h2o_e': 1, 'pyr_e':..."


### Metabolic Resource Overlap
**MRO** (metabolic resource overlap): calculates how much the species compete for the same metabolites.

In [44]:
score, MRO = mro_score(community,environment=M9)
print(score)
MRO

0.2857142857142857


,Value
Attribute,
community_medium,"{pi, o2, glu}"
individual_media,"{'glc_ko': {'pi', 'pyr', 'o2', 'nh4'}, 'nh4_ko..."


In [45]:
MRO.individual_media.glc_ko

{'nh4', 'o2', 'pi', 'pyr'}

In [46]:
MRO.individual_media.nh4_ko

{'glc', 'glu', 'pi'}